<a href="https://colab.research.google.com/github/fecsaba/FluxKiller/blob/main/FluxKiller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Flux killer**
### Notes
- Uncheck `UPDATE_COMFY_UI`, and `USE_COMFYUI_MANAGER` and `INSTALL_GGUF_NODE` after first run if `USE_GOOGLE_DRIVE` is checked.

#### Crystools

#### EasyUse
A ComfyUI használhatóságának javítása érdekében számos gyakran használt csomóponthoz optimalizáltak és integráltak.
#### ITools
Az iTools néhány életminőség-csomópont, mint például a kép létrehozásához használt lehetséges prompt beolvasása, a prompt fájlba mentése új sorként, a promptok beolvasása többsoros fájlból.
<!-- #### Was node suite
Csomópontcsomag a ComfyUI-hoz számos új csomóponttal, például képfeldolgozással, szövegfeldolgozással és egyebekkel.

#### ComfyUI's ControlNet Auxiliary Preprocessors
Plug-and-play ComfyUI csomópontkészletek ControlNet tippképek készítéséhez
"anime stílus, tiltakozás az utcán, cyberpunk város, egy rózsaszín hajú, arany szemű nő (nézi a nézőt) egy táblát tart a "ComfyUI ControlNet Aux" felirattal félkövér, neonrózsaszínnel a Flux.1 Dev-n
A kód másolás-beillesztése a https://github.com/lllyasviel/ControlNet/tree/main/annotator megfelelő mappáiból, és csatlakoztatva van a 🤗 Hubhoz .
#### ComfyUI ControlAltAI Nodes
Fluxus mintavevő
A Flux Sampler csomópont egyetlen, áramvonalas csomópontban egyesíti a CustomSamplerAdvance csomópont és a bemeneti csomópontok funkcióit.

CFG beállítás: A CFG fix értéke 1.
Kondicionálás bemenet: Csak pozitív kondicionálás támogatott.
Kompatibilitás: Csak a Flux modellel kompatibilis mintavevőket és ütemezőket tartalmazza.
Látens kompatibilitás: Csak SD3 üres látens képet használjon. A normál üres látens képcsomópont nem kompatibilis.

#### Rgthree Comfy
Csomópontok: Seed, Reroute, Context, Lora Loader Stack, Context Switch, Fast Muter. Ezek az egyéni csomópontok segítenek megszervezni az összetett munkafolyamatok felépítését.

## Florence2 in ComfyUI
A Florence-2 egy fejlett látásalap-modell, amely azonnali alapú megközelítést alkalmaz a látás és a látásnyelvi feladatok széles skálájának kezelésére. A Florence-2 képes értelmezni az egyszerű szöveges felszólításokat olyan feladatok végrehajtásához, mint a feliratozás, az objektumészlelés és a szegmentálás. Használja ki FLD-5B adatkészletünket, amely 5,4 milliárd megjegyzést tartalmaz 126 millió képen, hogy elsajátítsa a többfeladatos tanulást. A modell sorozatról-szekvenciára architektúrája lehetővé teszi, hogy mind a nulla lövés, mind a finomhangolt beállításokban kiváló legyen, így versenyképes látásalapmodellnek bizonyul. -->



# INIT

In [ ]:
# INIT
# #@title Environment Setup
from pathlib import Path
import os

OPTIONS = {}

USE_GOOGLE_DRIVE = False  #@param {type:"boolean"}
UPDATE_COMFY_UI = False  #@param {type:"boolean"}
USE_COMFYUI_MANAGER = True  #@param {type:"boolean"}
INSTALL_GGUF_NODE = False  #@param {type:"boolean"}
INSTALL_CUSTOM_NODES_DEPENDENCIES = True  #@param {type:"boolean"}

OPTIONS.update({
    'USE_GOOGLE_DRIVE': USE_GOOGLE_DRIVE,
    'UPDATE_COMFY_UI': UPDATE_COMFY_UI,
    'USE_COMFYUI_MANAGER': USE_COMFYUI_MANAGER,
    'INSTALL_GGUF_NODE': INSTALL_GGUF_NODE,
    'INSTALL_CUSTOM_NODES_DEPENDENCIES': INSTALL_CUSTOM_NODES_DEPENDENCIES
})

current_dir = !pwd
WORKSPACE = f"{current_dir[0]}/ComfyUI"

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /
    from google.colab import drive
    drive.mount('/content/drive')
    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

!export COMFYUI_PATH=WORKSPACE

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-
  !chmod 755 .ci/**/*.bat .ci/**/*.py .ci/**/*.txt
  !git pull

!echo -= Install dependencies =-
!pip3 install accelerate einops transformers>=4.28.1 safetensors>=0.4.2 aiohttp pyyaml Pillow scipy tqdm psutil tokenizers>=0.13.3
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip3 install torchsde kornia>=0.7.1 spandrel soundfile sentencepiece

if OPTIONS['USE_COMFYUI_MANAGER']:
  %cd custom_nodes
  !chmod 755 ComfyUI-Manager/**/*.sh
  ![ ! -d ComfyUI-Manager ] && echo -= Initial setup ComfyUI-Manager =- && git clone https://github.com/ltdrdata/ComfyUI-Manager
  %cd ComfyUI-Manager
  !git pull

if OPTIONS['INSTALL_GGUF_NODE']:
  %cd $WORKSPACE/custom_nodes
  !git clone https://github.com/city96/ComfyUI-GGUF
  !pip install -r ComfyUI-GGUF/requirements.txt

%cd $WORKSPACE

!echo -= Install custom nodes dependencies =-
!pip install GitPython
!python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies

!echo -= Install nodes =-
!echo -= **************** =-
!echo -= Install Crystools =-
%cd $WORKSPACE/custom_nodes
!git clone https://github.com/crystian/ComfyUI-Crystools.git
!pip install -r ComfyUI-Crystools/requirements.txt

!echo -= Install EasyUse =-
%cd $WORKSPACE/custom_nodes
!git clone https://github.com/yolain/ComfyUI-Easy-Use.git
!pip install -r ComfyUI-Easy-Use/requirements.txt


def install_custom_node(comfyui_path, repo_url, node_folder_name):
    custom_nodes_path = os.path.join(comfyui_path, "custom_nodes")

    if not os.path.exists(custom_nodes_path):
        os.makedirs(custom_nodes_path)
    print(f"custom_nodes mappa elérési útvonala: {custom_nodes_path}")

    destination_path = os.path.join(custom_nodes_path, node_folder_name)

    if not os.path.exists(destination_path):
        !git clone {repo_url} {destination_path}
        print(f"{node_folder_name} telepítése kész.")
    else:
        print(f"{node_folder_name} már telepítve van.")

    requirements_path = os.path.join(destination_path, "requirements.txt")
    if os.path.exists(requirements_path):
        !pip install -r {requirements_path}
        print("Dependenciák telepítve.")
    else:
        print("Nincs requirements.txt fájl, nem szükséges dependenciákat telepíteni.")

comfyui_path = "/content/drive/MyDrive/ComfyUI"
# # iTools
# repo_url = "https://github.com/fecsaba/ComfyUI-iTools.git"
# node_folder_name = "ComfyUI-iTools"

# install_custom_node(comfyui_path, repo_url, node_folder_name)

# # Was node suite
# repo_url = "https://github.com/WASasquatch/was-node-suite-comfyui.git"
# node_folder_name = "was-node-suite-comfyui"

# install_custom_node(comfyui_path, repo_url, node_folder_name)

# # ComfyUI's ControlNet Auxiliary Preprocessors
# repo_url = "https://github.com/Fannovel16/comfyui_controlnet_aux.git"
# node_folder_name = "comfyui_controlnet_aux"

# install_custom_node(comfyui_path, repo_url, node_folder_name)

# # ComfyUI ControlAltAI Nodes
# repo_url = "https://github.com/gseth/ControlAltAI-Nodes.git"
# node_folder_name = "ControlAltAI-Nodes"

# install_custom_node(comfyui_path, repo_url, node_folder_name)

# ComfyUI RgThree Nodes
repo_url = "https://github.com/rgthree/rgthree-comfy.git"
node_folder_name = "rgthree-comfy"

install_custom_node(comfyui_path, repo_url, node_folder_name)

# # Florence2
# repo_url = "https://github.com/kijai/ComfyUI-Florence2.git"
# node_folder_name = "ComfyUI-Florence2"

# install_custom_node(comfyui_path, repo_url, node_folder_name)

%cd $WORKSPACE
print("Telepítés befejezve. Indítsd újra a ComfyUI-t!")



# Load models

In [ ]:
### Download Resources
%cd $WORKSPACE

!wget -c "https://huggingface.co/SG161222/RealVisXL_V5.0_Lightning/resolve/main/RealVisXL_V5.0_Lightning_fp16.safetensors" -P "./models/checkpoints"
!wget -c "https://huggingface.co/SG161222/RealVisXL_V5.0_Lightning/blob/main/RealVisXL_V5.0_Lightning_fp32.safetensors" -P "./models/checkpoints"
!wget -c "https://huggingface.co/stabilityai/sdxl-vae/resolve/main/sdxl_vae.safetensors" -P "./models/vae"

# !mkdir -p "/content/models/"
# # SDXL model
# !wget -c "https://civitai.com/api/download/models/782002?type=Model&format=SafeTensor&size=full&fp=fp16" -O "/content/models/Juggernaut.safetensors"
# !ln -s "/content/models/Juggernaut.safetensors" "./models/checkpoints/Juggernaut.safetensors"

# # diffusion_pytorch_model_promax
# !wget -c "https://huggingface.co/xinsir/controlnet-union-sdxl-1.0/resolve/main/diffusion_pytorch_model_promax.safetensors" -O "/content/models/diffusion_pytorch_model_promax.safetensors"
# !ln -s "/content/models/diffusion_pytorch_model_promax.safetensors" "./models/checkpoints/diffusion_pytorch_model_promax.safetensors"

# # Download the FLUX model from Hugging Face and save it in the ./models/unet directory
# !wget -c "https://huggingface.co/city96/FLUX.1-dev-gguf/resolve/main/flux1-dev-Q4_K_S.gguf" -P "/content/models"
# !ln -s "/content/models/flux1-dev-Q4_K_S.gguf" "./models/unet/flux1-dev-Q4_K_S.gguf"
# # !ln -s "/models/flux1-dev-Q8_0.gguf" "./models/unet/flux1-dev-Q8_0.gguf"

# # Download the CLIP T5 model from Hugging Face and save it in ./models/clip
# !wget -c "https://huggingface.co/city96/t5-v1_1-xxl-encoder-gguf/resolve/main/t5-v1_1-xxl-encoder-Q4_K_S.gguf" -P "/content/models"
# !ln -s "/content/models/t5-v1_1-xxl-encoder-Q4_K_S.gguf" "./models/clip/t5-v1_1-xxl-encoder-Q4_K_S.gguf"
# # !ln -s "/models/t5-v1_1-xxl-encoder-Q8_0.gguf" "./models/clip/t5-v1_1-xxl-encoder-Q8_0.gguf"

# # Download the safetensors file for the CLIP model
# !wget -c "https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/clip_l.safetensors?download=true" -P "/content/models"
# !ln -s "/content/models/clip_l.safetensors" "./models/clip/clip_l.safetensors"
# # !ln -s "/content/models/clip_l-for-gguf.safetensors" "./models/clip/clip_l-for-gguf.safetensors"

# # Download the VAE model from Hugging Face and save it in ./models/vae
# !wget -c "https://huggingface.co/f5aiteam/VAE/resolve/main/ae.safetensors" -P "/content/models"
# !ln -s "/content/models/ae.safetensors" "./models/vae/ae.safetensors"
# # !ln -s "/content/models/ae.safetensors" "./models/vae/ae.safetensors"

# # Download the diffusion model from Hugging Face and save it in ./models/controlnet
# !wget -c "https://huggingface.co/Shakker-Labs/FLUX.1-dev-ControlNet-Union-Pro/resolve/main/diffusion_pytorch_model.safetensors?download=true" -P "/content/models"
# !ln -s "/content/models/diffusion_pytorch_model.safetensors?download=true" "./models/controlnet/diffusion_pytorch_model.safetensors"

# Download the LoRa model from Civitai.com Replace `298eebac3e9fecdcd7def0c8a41fa14d` with your actual token
# !wget -c "https://civitai.com/api/download/models/1198851?type=Model&format=SafeTensor&token=298eebac3e9fecdcd7def0c8a41fa14d" -O "./models/loras/FluxThouS40k.safetensors"
# !wget -c "https://civitai.com/api/download/models/1198851?type=Model&format=SafeTensor&token=298eebac3e9fecdcd7def0c8a41fa14d" -O "./models/loras/FluxThouS40k.safetensors"
# !wget -c "https://huggingface.co/city96/FLUX.1-dev-gguf/resolve/main/flux1-dev-Q4_K_S.gguf" -P "./models/unet"
# !wget -c "https://huggingface.co/city96/t5-v1_1-xxl-encoder-gguf/resolve/main/t5-v1_1-xxl-encoder-Q4_K_S.gguf" -P "./models/clip"
# !wget -c "https://huggingface.co/f5aiteam/CLIP/resolve/main/clip_l.safetensors" -P "./models/clip"
# !wget -c "https://huggingface.co/f5aiteam/VAE/resolve/main/ae.safetensors" -P "./models/vae"



%cd $WORKSPACE
# Download the VAE model from Hugging Face and save it in ./models/vae
# !wget -c "https://huggingface.co/f5aiteam/VAE/resolve/main/ae.safetensors" -P "/content/models"
# !ln -s "/content/models/ae.safetensors" "./models/vae/ae.safetensors"
# Download the diffusion model from Hugging Face and save it in ./models/controlnet
# !wget -c "https://huggingface.co/Shakker-Labs/FLUX.1-dev-ControlNet-Union-Pro/resolve/main/diffusion_pytorch_model.safetensors?download=true" -P "/content/models"
# !ln -s "/content/models/diffusion_pytorch_model.safetensors?download=true" "./models/controlnet/diffusion_pytorch_model.safetensors"
# Download the safetensors file for the CLIP model
# !wget -c "https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/clip_l.safetensors?download=true" -P "/content/models"
# !ln -s "/content/models/clip_l.safetensors?download=true" "./models/clip/clip_l.safetensors"
# Download the CLIP T5 model from Hugging Face and save it in ./models/clip
# !wget -c "https://huggingface.co/city96/t5-v1_1-xxl-encoder-gguf/resolve/main/t5-v1_1-xxl-encoder-Q5_K_M.gguf?download=true" -P "/content/models"
# !ln -s "/content/models/t5-v1_1-xxl-encoder-Q5_K_M.gguf?download=true" "./models/clip/t5-v1_1-xxl-encoder-Q5_K_M.gguf"


### Run ComfyUI

In [2]:
!wget -P ~ https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i ~/cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

--2025-01-27 18:32:39--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.1.0/cloudflared-linux-amd64.deb [following]
--2025-01-27 18:32:40--  https://github.com/cloudflare/cloudflared/releases/download/2025.1.0/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/106867604/2d8414e3-a676-4d22-b0ad-71fffb4149a4?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20250127%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20250127T183240Z&X-Amz-Expires=300&X-Amz-Signature=88c3644f7f41d17a1fdfce9d9e3ec0f31fa897dc25de51ffbbf0c1c9